## Libraries etc.

In [1]:
import dill as pickle
import csv
import re
import numpy as np
import networkx as nx
# #
from scipy.stats import fisher_exact
from scipy.sparse import csr_matrix
from datetime import datetime
from collections import Counter, defaultdict
from unidecode import unidecode
from editdistance import eval as ed
from itertools import chain, combinations
from evlex_shared_scripts import *

## Part 1: preparing data

#### A: read in data

In [2]:
# files
metadata_fn = './files/doreco_files_metadata.csv'
corpus_fn = './files/corpus_doreco.p'

In [3]:
genres = {'personal narrative','traditional narrative','conversation','procedural'}
#
raw_corpus = pickle.load(open(corpus_fn, 'rb'))    
corpus = select_genres(raw_corpus, metadata_fn, genres)

[('traditional narrative', 626), ('personal narrative', 436), ('stimulus retelling', 158), ('conversation', 108), ('procedural', 76), ('procedural/conversation', 2)]


In [4]:
wct = Counter()
for doc in corpus:
    for file in corpus[doc]:
        for line, elt in corpus[doc][file].items():
            wct[doc[:-4]] += len(elt['spc'])
with open('./files/word_counts.csv', 'w') as fh:
    fh.write('doculect,count\n' + '\n'.join('%s,%d' % i for i in wct.items()))

In [5]:
wct.most_common()

[('kama1351', 77445),
 ('bora1263', 53905),
 ('kaka1265', 53428),
 ('komn1238', 47393),
 ('goem1240', 46673),
 ('sout3282', 44201),
 ('nngg1234', 40876),
 ('beja1238', 38121),
 ('arap1274', 35839),
 ('dolg1241', 34863),
 ('trin1278', 31221),
 ('goro1270', 31186),
 ('urum1249', 29006),
 ('sout2856', 26969),
 ('yong1270', 26326),
 ('vera1241', 23973),
 ('bain1259', 22733),
 ('cash1254', 21875),
 ('pnar1238', 20796),
 ('ruul1235', 20133),
 ('anal1239', 20041),
 ('even1259', 19877),
 ('sanz1248', 19693),
 ('svan1243', 19313),
 ('orko1234', 19309),
 ('texi1237', 19218),
 ('nort2641', 18824),
 ('apah1238', 18726),
 ('nisv1234', 18217),
 ('kark1256', 17835),
 ('movi1243', 17308),
 ('teop1238', 16777),
 ('nort2875', 15818),
 ('tsim1256', 14529),
 ('port1286', 14281),
 ('jeju1234', 14169),
 ('lowe1385', 13814),
 ('jeha1242', 13060),
 ('sadu1234', 12286),
 ('resi1247', 11552),
 ('savo1255', 10897),
 ('hoch1243', 10729),
 ('sumi1235', 10345),
 ('yuca1254', 9936),
 ('taba1259', 9346),
 ('cabe1245'

## B: extract TEs

In [6]:
## fragment extraction

def get_fragments(w, min_len=2, max_len=7, yield_self=True):
    """
    takes a string *w* and extracts all substrings of minimal length *min_len* and maximal length *max_len*
    """
    if yield_self and len(w) > max_len: yield w
    for i in range(len(w)):
        for j in range(i+min_len, min(i+max_len+1,len(w)+1)):
            if (not (i == 0 or j == len(w))) or (j-i) >= min_len+1:
                yield w[i:j]

def get_all_fragments(F, split_words = True, frequency_threshold = 1):
    """
    given a document stored in a dictionary *tbd*, mapping an identifier key 
    onto a string containing the text, and a set of *all_verses*
    (the shared identifier keys between tbd and the source document(,
    this function returns a sparse matrix *fragments* of identifier key (rows) 
    to substrings of the text (column), with the matrix being True if the fragment
    occurs for that identifier key and False otherwise.
    as well as dictionaries for the identification
    of the rows and columns.
    (memory/computation efficient format, but a bit densely written)
    """
    wordcount = Counter((w for l in F for w in l))
    if split_words: 
        word_fragments = {w : set(get_fragments('^%s$' % unidecode(w).lower())) for w in wordcount.keys() }
    else: 
        word_fragments = {w : {unidecode(w).lower()} for w in wordcount.keys() }
    
    fragment_count = Counter((f for w,F in word_fragments.items() for f in F if f != '' for i in range(wordcount[w])))
    fragment_ixx = {f:i for i,(f,c) in enumerate(fragment_count.most_common()) if c >= frequency_threshold }
    #
    R,C = [], []
    for line_ix, line_f in enumerate(F):
        if len(line_f) == 0: continue
        line_frags = list(map(lambda f : fragment_ixx[f],
                              filter(lambda f : f in fragment_ixx,
                                     set.union(*map(lambda w : word_fragments[w], line_f)))))
        R.extend([line_ix]*len(line_frags))
        C.extend(line_frags)
    fragments = csr_matrix((np.ones(len(R)), (R,C)), dtype=bool, shape = (len(F),max(C)+1))
    return fragments, fragment_ixx

In [7]:
def create_alignments(doc, corpus, min_freq = 1):
    #
    E, F, I = zip(*((get_lexical(get_mappable(e['spc'])), e['tx'].split(), (f,l))
                    for f in corpus[doc] for l,e in corpus[doc][f].items()))
    #for e,f,i in zip(E,F,I[:5]): print(i,'\n\t',e,'\n\t',f)
    e_fragments, e_dic = get_all_fragments(E, split_words=False, frequency_threshold=1)
    e_counts = Counter([unidecode(e).lower() for l in E for e in l if e != ''])
    e_seed = {e : e_dic[e] for e in e_counts if e_counts[e] >= min_freq and e != ''}
    f_fragments, f_dic = get_all_fragments(F, split_words=True, frequency_threshold=1)
    f_list = np.array(sorted(f_dic, key = lambda k : f_dic[k]))
    #
    # get TEs
    tes, te_words = {}, {}
    for e,ei in sorted(e_seed.items(), key = lambda x : -e_counts[x[0]]):
        pos = e_fragments[:,ei].nonzero()[0]
        tes[e] = extract_tes(pos, f_fragments, f_dic, coverage=.95, min_trans=0.01, min_backtrans=0.10)
        #tes[e] = merge_similar_tes(tes[e])
        te_words[e] = { te : get_te_words(f_fragments, f_list, te, te_pos) for te,te_pos in tes[e].items() }
        # print(e, ei, e_counts[e], len(pos),'\n\t', tes[e].keys(), te_words[e])
    print(doc, datetime.now(), len(tes))
    pickle.dump((tes, te_words), open('./pickles/tes_%s.p'%doc,'wb'))

In [8]:
def extract_tes(pos_ixx, fragments, fixx, coverage = 0.95, min_trans = 0.05, min_backtrans = 0.25, verbose=False):
    """
    implements one forward pass of the Liu et al. (2023) approach.
    Takes a list *pos* of positive instances (row identifiers for fragments)
    As well as the sparse matrix *fragments* and the two dictionaries
    (vixx -- verse_ixx and fixx -- fragment_ixx).
    The parameters determine the fragments considered in the extraction: the iteration keeps going until
    either no good fragments can be found (significance under .001) or the *coverage* has been reached.
    *min_trans* is a float [0,1] that filters out all fragments that occur in hit verses in less than min_trans
    proportion of all hit verses.
    """
    neg_ixx = list(set(range(fragments.shape[0]))-set(pos_ixx))
    flist = [None] * len(fixx)
    for k,v in fixx.items():
        flist[v] = k
    #sorted(fixx, key = lambda k : fixx[k])
    #
    pos_tot_orig = pos_tot = len(pos_ixx)
    neg_tot = len(neg_ixx)
    #
    pos_ct = fragments[pos_ixx].sum(0).A[0]
    neg_ct = fragments[neg_ixx].sum(0).A[0]
    #
    good_fragments = np.where((pos_ct >= 1) & ((pos_ct/pos_tot_orig) >= min_trans) &
                             (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
    string_props = [(f[0] == '^', f[-1] == '$', len(f)) for f in flist]
                    #len(re.sub('[.*]', '', f)))
    #
    hits = defaultdict(lambda : [])
    ct = 0
    fe_scores = {}
    while len(pos_ixx) >= (1-coverage) * pos_tot_orig:
        ct += 1
        #
        # GET BEST
        assoc_scores = Counter()
        for f in good_fragments:
            table = ((pos_ct[f],pos_tot-pos_ct[f]),(neg_ct[f],neg_tot-neg_ct[f]))
            try: fe_score = fe_scores[table]
            except KeyError: fe_score = fe_scores[table] = -np.log(fisher_exact(table, alternative='greater')[1])
            assoc_scores[f] = (fe_score, string_props[f])
        best, best_score = next((x for x in assoc_scores.most_common(1)),(None,None))
        if best == None or best_score[0] < -np.log(5e-2):
            break
        # print([flist[k] for k,v in assoc_scores.most_common(10)])
        #
        # UPDATE
        new_pos_ixx = []
        for pos_v in pos_ixx:
            if fragments[pos_v,best] > 0: hits[flist[best]].append(pos_v)
            else: new_pos_ixx.append(pos_v)
        #
        pos_ixx = new_pos_ixx
        pos_tot = len(pos_ixx)
        pos_ct = fragments[pos_ixx].sum(0).A[0]
        neg_ct = fragments[neg_ixx].sum(0).A[0]
        #
        good_fragments = np.where((pos_ct >= 1) & (pos_ct/pos_tot_orig >= min_trans) &
                                    (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
        if verbose: print(ct, flist[best])
    return hits

def merge_similar_tes(tes):
    merge = nx.Graph()
    for t1, t2 in combinations(tes,2):
        if t1 in t2 or t2 in t1 or ed(t1,t2) <= 1:
            merge.add_edge(t1,t2)
    for c in nx.connected_components(merge):
        if len(c) >= 2:
            best_c = max(c, key= lambda k : len(tes[k]))
            tes[best_c] = list(chain(*[tes[ci] for ci in c]))
            for ci in filter(lambda ci : ci != best_c, c): del tes[ci]
            #print('>>> MERGED', c)
    return tes

def longest_nonoverlapping(m, rev, frags):
    frags = [rev[f] for f in frags]
    Ma = [mi for mi in frags if m in mi and 
          next((False for mj in frags if mj != mi and mi in mj),True)]
    return Ma

def get_te_words(fragments_F, rev, te, te_pos):
    ctr = Counter()
    for pos in te_pos:
        frags = fragments_F[pos].nonzero()[1]
        Ma = longest_nonoverlapping(te, rev, frags)
        for m in Ma:
            ctr[m] += 1
    return ctr

In [9]:
for doc in corpus:
    create_alignments(doc, corpus)

/tmp/ipykernel_1515612/1182429609.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]


anal1239.csv 2024-10-28 15:49:34.897733 1125
apah1238.csv 2024-10-28 15:49:44.041965 779
arap1274.csv 2024-10-28 15:50:08.026997 953
bain1259.csv 2024-10-28 15:50:27.626472 1167
beja1238.csv 2024-10-28 15:51:07.188586 1247
bora1263.csv 2024-10-28 15:52:20.130860 1815
cabe1245.csv 2024-10-28 15:52:25.844092 457
cash1254.csv 2024-10-28 15:52:38.408719 883
dolg1241.csv 2024-10-28 15:53:11.050525 1343
even1259.csv 2024-10-28 15:53:32.183058 948
goem1240.csv 2024-10-28 15:53:40.005157 893
goro1270.csv 2024-10-28 15:53:56.814768 1065
hoch1243.csv 2024-10-28 15:54:04.291441 558
jeha1242.csv 2024-10-28 15:54:08.221180 485
jeju1234.csv 2024-10-28 15:54:17.697955 777
kaka1265.csv 2024-10-28 15:54:43.425064 1633


/tmp/ipykernel_1515612/1182429609.py:40: RuntimeWarning: divide by zero encountered in log
  except KeyError: fe_score = fe_scores[table] = -np.log(fisher_exact(table, alternative='greater')[1])


kama1351.csv 2024-10-28 15:55:22.891528 1336
kark1256.csv 2024-10-28 15:55:35.528618 812
komn1238.csv 2024-10-28 15:56:12.826234 1366
lowe1385.csv 2024-10-28 15:56:21.064941 863
movi1243.csv 2024-10-28 15:56:32.208920 825
ngal1292.csv 2024-10-28 15:56:35.100712 358
nisv1234.csv 2024-10-28 15:56:43.218431 724
nngg1234.csv 2024-10-28 15:56:53.716290 920
nort2641.csv 2024-10-28 15:57:03.368138 980
nort2875.csv 2024-10-28 15:57:13.506871 881
orko1234.csv 2024-10-28 15:57:19.389148 646
pnar1238.csv 2024-10-28 15:57:31.424915 1108
port1286.csv 2024-10-28 15:57:35.619906 561
resi1247.csv 2024-10-28 15:57:42.819715 509
ruul1235.csv 2024-10-28 15:58:03.036831 1162
sadu1234.csv 2024-10-28 15:58:11.377139 942
sanz1248.csv 2024-10-28 15:58:40.838646 1105
savo1255.csv 2024-10-28 15:58:45.284982 542
sout2856.csv 2024-10-28 15:58:56.545013 848
sout3282.csv 2024-10-28 15:59:23.088508 1968
sumi1235.csv 2024-10-28 15:59:27.458430 552
svan1243.csv 2024-10-28 15:59:47.754157 1150
taba1259.csv 2024-10-28 1

## post-hoc merging

In [10]:
import networkx as nx
from itertools import combinations, chain
from editdistance import eval as ed
from scipy.stats import entropy
from numpy.linalg import norm
import numpy as np

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

def merge_similar_tes_within(tes, te_words):
    merge = nx.Graph()
    merge.add_nodes_from(tes)
    tes_new, te_words_new = {}, {}
    for t1, t2 in combinations(tes,2):
        if t1 in t2 or t2 in t1 or ed(t1,t2) <= 1:
            merge.add_edge(t1,t2)
    for c in nx.connected_components(merge):
        best_c = max(c, key= lambda k : len(tes[k]))
        tes_new[best_c] = list(chain(*[tes[ci] for ci in c]))
        te_words_new[best_c] = sum([te_words[ci] for ci in c], Counter())
        #print('>>> MERGED', c)
    return tes_new, te_words_new

def merge_similar_tes_across(tes, te_words):
    merge_markers = nx.Graph()
    te_marker_pairs = [(te, marker) for te in tes for marker in tes[te] if len(tes[te][marker]) >= 5]
    for (tei,mrki),(tej,mrkj) in combinations(te_marker_pairs,2):
        if tei==tej: continue
        if set(te_words[tei][mrki]) & set(te_words[tej][mrkj]) != set():
            mrkic, mrkjc = mrki.strip('^$'), mrkj.strip('^$')
            minlen = min(len(mrkic),len(mrkjc))
            wi = te_words[tei][mrki]
            wj = te_words[tej][mrkj]
            jsd = (JSD(*zip(*[[wi[x],wj[x]] for x in sorted(set(wi)|set(wj))])))
            cos = (wv.wv.similarity(tei,tej))
            jac = len(set(wi)&set(wj))/len(set(wi)|set(wj))
            form = mrkic[:minlen] == mrkjc[:minlen] or mrkic[-minlen:] == mrkjc[-minlen:]
            if cos >= 2/3 and jsd <= 2/3 and form:
                #print((tei,mrki),(tej,mrkj),'JSD=%.2f COS=%.2f JAC=%.2f FORM=%d' % (jsd, cos, jac,form))
                merge_markers.add_edge((tei,mrki), (tej,mrkj))
    for c in nx.connected_components(merge_markers):
        new_mrk = min([mrk for te,mrk in c],key = len)
        for te,mrk in c:
            tes[te][new_mrk] = tes[te][mrk]
            te_words[te][new_mrk] = te_words[te][mrk]
            if mrk != new_mrk: del tes[te][mrk], te_words[te][mrk]
    return tes, te_words

def merge_similar_tes(doc):
    tes, te_words = pickle.load(open('./pickles/tes_%s.p' % doc,'rb'))
    for k,v in tes.items():
        tes[k], te_words[k] = merge_similar_tes_within(v, te_words[k])
    #
    tes, te_words = merge_similar_tes_across(tes, te_words)
    return tes, te_words